## Exercise 5: Probabilistic Seismic Hazard Analysis (PSHA) for Lampahan Region

This notebook demonstrates a workflow for estimating earthquake hazard at the building level using open data and reproducible Python tools. The main steps include:

- **Defining the Area of Interest:**  
    The Lampahan region is specified using a bounding box, and its geometry is visualized.

- **Downloading Building Footprints:**  
    Building polygons are retrieved from OpenStreetMap (OSM) and plotted on the map.

- **Fetching Earthquake Catalog:**  
    Earthquake events since 1980 are downloaded from the USGS using the ObsPy FDSN client, filtered by magnitude and spatial extent.

- **Computing Distances:**  
    Epicentral distances from each building centroid to each earthquake are calculated using the haversine formula.

- **Ground Motion Prediction:**  
    The Atkinson & Boore (2003) ground motion model (GMPE) is used to estimate the median peak ground acceleration (PGA) at each building for each event.

- **Hazard Curve Calculation:**  
    For each building, the mean annual frequency of exceedance (MAFE) for several PGA levels is computed, assuming each event is a Poisson process.

- **Probability Mapping:**  
    The probability that PGA exceeds 0.2g in 50 years is mapped for all buildings, visualized both as a static plot and interactively with Folium.

This exercise illustrates how open-source geospatial and seismological libraries can be combined for rapid, transparent hazard mapping at high spatial resolution. The workflow is modular and can be adapted for other regions, building datasets, or ground motion models.

The following code cell ensures that all required Python packages for seismic hazard analysis are installed and available in the notebook environment. It checks for each package, installs any missing ones using `pip`, and imports them for use in subsequent cells. This step helps maintain reproducibility and prevents runtime errors due to missing dependencies.

In [ ]:
#%% Install packages
# 1) Install dependencies if missing
import sys, subprocess, importlib

def ensure(pkg, import_name=None):
    name = import_name or pkg
    try:
        importlib.import_module(name)
        print(f"[ok] {pkg} already installed")
    except Exception:
        print(f"[info] Installing {pkg} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
        importlib.import_module(name)
        print(f"[ok] {pkg} installed")


pkgs = ["os", "re", "requests", "zipfile", "geopandas",
 "matplotlib", "shapely", "cartopy", "obspy", "folium"]

for pkg in pkgs:
    ensure(pkg)

# Import required libraries

In the next cell, we will import the required packages and define the parameters and constants needed for the exercise. We will also define functions that will be used later in the calculations.

In [ ]:
#%% Imports and constants
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box, Point
import osmnx as ox
from math import radians, sin, cos, asin, sqrt, log10, log, exp
from datetime import datetime
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import MarkerCluster
from branca.colormap import linear

from obspy.clients.fdsn import Client
from obspy.core.utcdatetime import UTCDateTime
from obspy.geodetics import gps2dist_azimuth
from math import log, sqrt
from scipy.stats import norm

ox.settings.log_console = False
ox.settings.use_cache = True

# Constants
R_EARTH_KM = 6371.0  # Earth radius in kilometers

# Prefer the `features` API (features_from_polygon) as shown in the Python-GIS guide.
features_from_polygon = ox.features.features_from_polygon

# Lampahan bounding box (degrees)
min_lat, max_lat = 4.70, 4.80
min_lon, max_lon = 96.70, 96.85

# Create GeoDataFrame for the region (WGS84)
region_poly = box(min_lon, min_lat, max_lon, max_lat)
gdf_region = gpd.GeoDataFrame({"name": ["Lampahan_box"]}, geometry=[region_poly], crs="EPSG:4326")

ax = gdf_region.plot(facecolor="none", edgecolor="red", linewidth=2, figsize=(10,10))
plt.title("Lampahan teaching region (hard-coded)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

# Downloading Building Footprints in the Aceh Lampahan Region

In the next section, we will download the building footprints within the area of interest (AOI).

In [ ]:
# %% Download building footprints from OSM
tags = {"building": True}
# Use the compatibility wrapper to fetch features (buildings) for the polygon AOI
buildings = features_from_polygon(region_poly, tags)
# Keep only polygons (some entries could be points/lines)
buildings = buildings[buildings.geometry.type.isin(["Polygon", "MultiPolygon"])].copy()
buildings = buildings.to_crs("EPSG:4326")

print(f"Downloaded {len(buildings)} building geometries from OSM.")
buildings.head(3)

#%% plot buildings
ax = gdf_region.plot(facecolor="none", edgecolor="red", linewidth=2, figsize=(10,10))
buildings.plot(ax=ax, facecolor="lightgrey", edgecolor="black", alpha=0.7)
plt.title("Building footprints from OSM in Lampahan region")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

# Earthquake Catalog and Distance Calculation

In the following cell, we download the earthquake catalog for the Lampahan region using the USGS FDSN client via ObsPy. The catalog includes events since 1980 with magnitude ≥ 5.0. For each earthquake, we extract the origin time, magnitude, latitude, longitude, and depth. We then compute the epicentral distance from each building centroid to each earthquake using the haversine formula. This distance matrix is essential for subsequent ground motion and hazard calculations.

In [ ]:
# %% Download earthquake catalog from USGS
# Using obspy's FDSN client interface
client = Client("USGS")
t1 = UTCDateTime("1980-01-01")
t2 = UTCDateTime()
minmag = 5.0

# Expand the box a bit for sources just outside
pad = 0.5
cat = client.get_events(starttime=t1, endtime=t2, minmagnitude=minmag,
                        minlatitude=min_lat - pad, maxlatitude=max_lat + pad,
                        minlongitude=min_lon - pad, maxlongitude=max_lon + pad)

print(f"Fetched {len(cat)} earthquakes (USGS FDSN).")


# %%
def haversine(lat1, lon1, lat2, lon2):
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    # Haversine formula
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = R_EARTH_KM * c
    return km

rows = []
for ev in cat:
    o = ev.preferred_origin() or ev.origins[0]
    m = ev.preferred_magnitude() or ev.magnitudes[0]
    rows.append({
        "time": o.time.datetime,
        "Mw": float(m.mag),
        "lat": float(o.latitude),
        "lon": float(o.longitude),
        "depth_km": float((o.depth or 0.0)/1000.0),
        "id": ev.resource_id.id
    })

df_ev = pd.DataFrame(rows).sort_values("time").reset_index(drop=True)
df_ev.tail(3)

## Ground Motion and Hazard Curve Calculation

In the next cell, we compute the seismic hazard for each building using the Atkinson & Boore (2003) ground motion model. For every building and earthquake event, the median peak ground acceleration (PGA) is estimated, and the probability of exceeding several PGA levels is calculated assuming lognormal variability. These probabilities are combined to produce a hazard curve for each building, and the probability of exceeding 0.2g in 50 years is mapped across the region. The results are visualized using a color-coded map to highlight spatial variations in seismic risk.

In [ ]:
# %% 4) Compact AB03 (2003) PGA (g) function for slab earthquakes
# - Interface vs In-slab via depth threshold (50 km).
# - Distance ∼ epicentral distance (teaching simplification).
# - We'll also define a helper to compute exceedance probability for a given PGA level assuming 
# a **lognormal** scatter with a single sigma in natural log (e.g., 0.6).
#
# **Reference**: Atkinson & Boore (2003) subduction GMPE (PGA). 
# (Coefficients and functional form as used earlier in this session.)
# AB03 PGA coefficients (Table 1) for compact implementation
AB03_PGA = {
    "interface": dict(c1= 2.99100,  c2=0.03525, c3=0.00759,  c4=-0.00206,  c5=0.19, c6=0.24, c7=0.29),
    "inslab":    dict(c1=-0.04713,  c2=0.60599, c3=0.01130,  c4=-0.000228, c5=0.19, c6=0.24, c7=0.29),
}

def g_slope(M, etype):
    return 10.0**(1.2 - 0.18*M) if etype=="interface" else 10.0**(0.301 - 0.01*M)

def D_of_M(M):  # km
    return 0.00724 * (10.0 ** (0.507*M))

def pga_ab03(Mw, depth_km, R_epi_km, site_class="B"):
    etype = "interface" if depth_km < 50.0 else "inslab"
    M = min(Mw, 8.5 if etype=="interface" else 8.0)
    h = min(depth_km, 100.0)
    R = np.sqrt(R_epi_km**2 + D_of_M(M)**2)
    C = AB03_PGA[etype]
    SC = 1.0 if site_class.upper()=="C" else 0.0
    SD = 1.0 if site_class.upper()=="D" else 0.0
    SE = 1.0 if site_class.upper()=="E" else 0.0
    log10Y = C["c1"] + C["c2"]*M + C["c3"]*h + C["c4"]*R + \
             g_slope(M, etype)*np.log10(R) + C["c5"]*SC + C["c6"]*SD + C["c7"]*SE
    Y_cms2 = 10.0**log10Y
    return Y_cms2/981.0  # g


def prob_exceed_given_event(pga_level_g, median_g, sigma_ln=0.6):
    """
    P(IM > y | event) for a lognormal IM with ln-median=ln(median_g) and sigma_ln.
    """
    mu = np.log(median_g)
    z = (np.log(pga_level_g) - mu) / sigma_ln
    return 1.0 - norm.cdf(z)

# Get cell centroids and compute matric of epicentral distances.

This cell prepares the building centroid points and calculates the epicentral distance from each building to every earthquake event in the catalog. The resulting distance matrix (`R_be`) is essential for estimating ground motion at each building location during each event, which is a key step in probabilistic seismic hazard analysis (PSHA).

In [ ]:
# %% 5) Prepare building points and compute distances to each event
# We take **centroids** of building polygons as representative building locations.
# Building centroids
build_pts = buildings.copy().to_crs("EPSG:32647")  # UTM zone 47N
build_pts["geometry"] = build_pts.geometry.centroid
build_pts = build_pts[["geometry"]].reset_index(drop=True)
build_pts = gpd.GeoDataFrame(build_pts, geometry="geometry", crs="EPSG:4326")

# Extract lat/lon arrays for speed
b_lats = build_pts.geometry.y.values
b_lons = build_pts.geometry.x.values

# Event arrays
ev_lats = df_ev["lat"].values
ev_lons = df_ev["lon"].values
ev_Mw   = df_ev["Mw"].values
ev_dep  = df_ev["depth_km"].values

# Compute matrix of epicentral distances (buildings x events)
def epi_matrix_km(b_lats, b_lons, ev_lats, ev_lons):
    B, E = len(b_lats), len(ev_lats)
    R = np.zeros((B, E), dtype=float)
    for i in range(B):
        for j in range(E):
            d_km = haversine(b_lats[i], b_lons[i], ev_lats[j], ev_lons[j])
            R[i, j] = d_km
    return R

R_be = epi_matrix_km(b_lats, b_lons, ev_lats, ev_lons)
R_be.shape

# PSHA calculation

This cell performs a probabilistic seismic hazard analysis (PSHA) for each building in the Lampahan region. It calculates the mean annual frequency of exceedance (MAFE) for several peak ground acceleration (PGA) levels using the earthquake catalog and the Atkinson & Boore (2003) ground motion model. The probability that each building will experience PGA greater than 0.2g in the next 50 years is computed and mapped. The results are visualized as a color-coded map, highlighting spatial variations in seismic risk across the building stock.

In [ ]:
# ## Toy example for PSHA at buildings
# - Catalog spans from `t1` to `t2` → duration **T_years**.
# - Treat each event as a Poisson process with annual rate **1/T_years**.
# - For a PGA level `y`, the **mean annual frequency of exceedance (MAFE)** at a building is:
#   \n
#   `lambda(y) = sum_j [ (1/T) * P(IM > y | event j at that building) ]`
#   \n
# - Convert to **probability in N years**: `P_N = 1 - exp(-lambda(y) * N)`.
#
# We'll compute a small **hazard curve** for each building and also a single map value:
# **P( PGA > 0.2 g in 50 yrs )** at **Site Class = C** (changeable).
T_years = (t2.datetime - t1.datetime).days / 365.25
print(f"Catalog duration T ≈ {T_years:.1f} years")

# Settings
site_class_default = "C"     # try "B", "C", "D", "E"
sigma_ln_default   = 0.6     # pedagogical choice
pga_levels_g       = np.array([0.05, 0.10, 0.20, 0.30])  # hazard curve levels
N_years_for_map    = 50
pga_map_level      = 0.20

B, E = R_be.shape
hazard_mafe = np.zeros((B, len(pga_levels_g)), dtype=float)

# Precompute medians per building/event for speed
median_g = np.zeros((B, E), dtype=float)
for i in range(B):
    for j in range(E):
        median_g[i, j] = pga_ab03(ev_Mw[j], ev_dep[j], R_be[i, j], site_class=site_class_default)

# Sum rates
event_rate = 1.0 / T_years
for k, y in enumerate(pga_levels_g):
    # P(IM>y | event j) at building i
    pij = 1.0 - norm.cdf((np.log(y) - np.log(median_g)) / sigma_ln_default)
    # lambda_i(y) = sum_j rate * pij
    hazard_mafe[:, k] = event_rate * np.sum(pij, axis=1)

# Choose the mapping metric: P_N for y=0.2 g
idx = list(pga_levels_g).index(pga_map_level)
P50 = 1.0 - np.exp(-hazard_mafe[:, idx] * N_years_for_map)

# Attach to buildings GeoDataFrame
gdf_b = buildings.copy()
gdf_b["PGA_level_g"] = pga_map_level
gdf_b["P_exceed_50yr"] = P50
gdf_b = gdf_b.set_geometry(gdf_b.geometry).to_crs("EPSG:4326")

gdf_b[["PGA_level_g", "P_exceed_50yr"]].head()

# Plot map
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
gdf_region.boundary.plot(ax=ax, color="black", linewidth=1)
# Color ramp caps to keep the map readable
vals = gdf_b["P_exceed_50yr"].clip(0, 0.5)   # cap at 50%
gdf_b.plot(ax=ax, column=vals, cmap="viridis", linewidth=0,
           legend_kwds={"label": f"P(PGA > {pga_map_level:.2f} g in {N_years_for_map} yrs)",
                        "shrink": 0.6}, vmax=1.0, vmin=0.0)
plt.colorbar(ax.collections[0], ax=ax, fraction=0.03, pad=0.04)
ax.set_title(f"(Lampahan buildings — P(PGA > {pga_map_level:.2f}) g in {N_years_for_map} yrs), site={site_class_default})")
ax.set_xlabel("Longitude"); ax.set_ylabel("Latitude")
plt.show()

# Interactive Seismic Hazard Mapping with Folium

This cell creates an interactive map of the Lampahan region using Folium, displaying the probability that each building will experience peak ground acceleration (PGA) greater than 0.2g in the next 50 years. Buildings are color-coded according to their exceedance probability, with a capped color scale for readability. Users can zoom and pan to explore spatial patterns of seismic risk at the building level. Tooltips provide detailed probability values for each building footprint.

In [ ]:
#%% Plot on folium map (for interactive zoom/pan)
# Center of the region
center_lat = 0.5 * (min_lat + max_lat)
center_lon = 0.5 * (min_lon + max_lon)  
m = folium.Map(location=[center_lat, center_lon], zoom_start=13, tiles="cartodbpositron")
# Color map
colormap = linear.viridis.scale(0.0, 0.5)  # cap at 50%
colormap.caption = f"P(PGA > {pga_map_level:.2f} g in {N_years_for_map} yrs)"
colormap.add_to(m)
# Add buildings
marker_cluster = MarkerCluster().add_to(m)
for _, row in gdf_b.iterrows():
    p = row["P_exceed_50yr"]
    color = colormap(p) if p <= 0.5 else "#800026"  # dark red for >50%
    folium.GeoJson(
        row["geometry"],
        style_function=lambda feature, color=color: {
            "fillColor": color,
            "color": color,
            "weight": 0,
            "fillOpacity": 0.7,
        },
        tooltip=(f"P(PGA > {pga_map_level:.2f} g in {N_years_for_map} yrs): {p:.3f}"),
    ).add_to(marker_cluster)

m